## Manually calculate the LogFC for each replicate with it's controls

### This will create a table with each replicates replicate having a column.
### If we have 3 replicates for a chemical and 2 controls the output will have 3 columns one for each
### replicate containing the log2FC manually calculated by 

### log2FC = log2(  (chemical count)/(average control count) )

### Only significant gene values will be retained. Significance measurement comes from the EdgeR FDR score.
### If only one control has counts that will be used instead of the average of the counts


Nov. 6th 2019 

In [ ]:
import glob
import math
import os
import pandas as pd
import re

# read in the experiment information
sampleFile = '/mnt/4a00c042-1850-4128-bf06-17c0406f98ad/projects/glbrcChemicalGenomics/Nov_12_2018/YeastChemGen003-008_BarcodesToConditions_111718.csv'

### Get the sample and control matching information

In [ ]:
# get experiment information 
seqID = {}        # dictionary to hold experiment information

with open(sampleFile, 'r') as f:
    f.readline()                                                   # skip header
    for line in f:
        sample = line.rstrip().split('\t')
        chemical      = re.sub(r'\W','',sample[4])                 # remove all word char
        experiment    = re.sub(r'\W', '', sample[1])
        sampleBarcode = sample[6]                                  # looks like "GACAGTTGAC"
        experimentKey = experiment + '_' + chemical + '.' + sampleBarcode   
               
        if 'ctrl' in chemical:
            continue
        else:          
            if experimentKey not in seqID:             # track barcode
                    seqID[experimentKey] = []     
                    seqID[experimentKey].append(sample[7]) # add control 1
                    seqID[experimentKey].append(sample[8]) # add control 2
            else:
                print('Duplicate barcode for {}  {}'.format(experimentKey, sampleBarcode))  # no duplicate BC's allowed

# get number of significant genes, this file has chemical name and the number of significant genes 
cntLst = []
with open('../significant_gene_count_by_sample.csv', 'r') as f:
    f.readline()
    for ln in f:
        nLst = ln.rstrip().split('\t')
        nLst[1] = int(nLst[1])               
        cntLst.append(tuple(nLst))       

f.close()

# order the list of tuples (chemicalname, numSignificantGenes) from lowest to highest
orderedCntLst = sorted(cntLst, key=lambda tup: tup[1])


In [ ]:
# add these manually as they were processed separately
seqID['ChemGen005_MMS-A.AGAGCGTTCT'] = ['ChemGen005_ctrl_1', 'ChemGen005_ctrl_3']
seqID['ChemGen006_MMS-BC.GCTAGTCACT'] = ['ChemGen006_ctrl_1', 'ChemGen006_ctrl_2']
seqID['ChemGen006_MMS-BC.TAACCGGTAG'] = ['ChemGen006_ctrl_1', 'ChemGen006_ctrl_2']

seqID['ChemGen006_QUADRIS1.AGGCTTGACT'] = ['ChemGen006_ctrl_1','ChemGen006_ctrl_2']
seqID['ChemGen006_QUADRIS1.GGCGATATCA'] = ['ChemGen006_ctrl_1','ChemGen006_ctrl_2']
seqID['ChemGen006_QUADRIS1.TACAACGGTC'] = ['ChemGen006_ctrl_1','ChemGen006_ctrl_2']

seqID['ChemGen006_QUADRIS2.AGCTTGCGAA'] = ['ChemGen006_ctrl_1','ChemGen006_ctrl_2']
seqID['ChemGen006_QUADRIS2.GTTGTGAACC'] = ['ChemGen006_ctrl_1','ChemGen006_ctrl_2']
seqID['ChemGen006_QUADRIS2.TCAGTGCATG'] = ['ChemGen006_ctrl_1','ChemGen006_ctrl_2']

### Get the list of significant genes for each chemical
#### put in dictionary significantGenes

In [ ]:
sigFile = "/home/mplace/projects/glbrcChemicalGenomics/Nov_12_2018/edgeR_significantGenes"
os.chdir(sigFile)        # move to directory with significant gene list 

significantGenes = {}    # key = chemical name, value = list of significant genes

# loop through significant gene files
for sig in glob.glob("*_genelist.txt"):
    with open(sig, 'r') as f:
        chemName = re.sub('_genelist.txt', '', sig)   # extract chemical name
        significantGenes[chemName] = []               # create a list in dictionary
        for gene in f:
            gene = gene.rstrip()
            significantGenes[chemName].append(gene)   # add every gene name to list
    f.close()

In [ ]:
# read in Normalized data, this data was normalized in edgeR as a single table
os.chdir('/home/mplace/projects/glbrcChemicalGenomics/Nov_12_2018/calcNormTest')
data = pd.read_csv('dataEdgeR_normalized.csv', index_col = 0, sep="\t")

In [ ]:
#math.log(x.iloc[0]/((sum(x[1:3]))/2),2)
def log2FC(row):
    # if chemical count is zero
    if row[0] == 0:
        return "NA"
    # if both controls are zero
    if row[1] == 0 and row[2] == 0:    
        return "NA"
    # if first control is zero
    elif row[1] == 0:      
        return math.log(row[0]/row[2], 2)
    # if second control is zero
    elif row[2] == 0:
        return math.log(row[0]/row[1], 2)
    # situaltion normal 
    else:
        return math.log(row[0]/((row[1] + row[2])/2), 2)

In [ ]:
# this step takes a minute or two
dfLst = []   # hold a list of single column dataframes with the log2 ratio
for k in seqID.keys():
    chemName = k.split('.')[0]
    name     = chemName.split('_')[1]
    pattern = '{}|{}|{}'.format(k, seqID[k][0], seqID[k][1])   # get the chemgen00#_chemialname and controls
    tmpDF = data.filter(regex=pattern)  # search for the correct 3 columns
    tmpDF = tmpDF.loc[significantGenes[name]]
    # calculate the log2 of the replicate/(Avg of control counts)
    # skip chemicals that have no significant genes
    if len(tmpDF) != 0:
        tmpDF[ chemName + '_log2_ratio'] = tmpDF.apply(log2FC, axis=1)
        dfLst.append(tmpDF[chemName + '_log2_ratio'])
        #dfLst.append(tmpDF)
    
# join data frames
result = pd.concat(dfLst, axis=1, sort=True)

result.to_csv('log2FC_output_test.csv')

In [ ]:
# need to reorder the columns from lowest number of significant genes to highest 
columnOrder = []        # lowest to highest number of significant genes
for chem in orderedCntLst:
    columnOrder.append(chem[0])

# get current experiment name header information
resultHeader = list(result.columns)

In [ ]:
# reorder column names by match chemGen00#_name to the ordered chemical name in  columnOrder
newColumnOrder = []
for word in columnOrder:
    pattern = r'ChemGen00\d_' + re.escape(word) + '_log2_ratio'
    for col in resultHeader:
        match = re.search(pattern, col, re.IGNORECASE)   
        if match:
            newColumnOrder.append(match.group())
newColumnOrder

            
# apply the column order to get a new dataframe
outputDF = result[newColumnOrder]

# write to file
#FOR SOME REASON A FEW COLUMNS ARE DUPLICATED? CLEANED UP MANUALLY
outputDF.to_csv('log2FC_data.csv', sep='\t', na_rep = "NA")

In [ ]:
# read in the log2FC_data.csv, then subset data by positive and negative
# write positive and negative log2FC values to separate files.
lgFC = pd.read_csv('log2FC_data.csv', index_col = 0, sep="\t")

positive = lgFC[lgFC > 0]                                             # select all values greater than zero
positive.fillna(value="NA", inplace=True)
positive.to_csv('log2FC_positive_data.csv', sep='\t', na_rep="NA")
negative = lgFC[lgFC < 0]
negative.fillna(value="NA", inplace=True)
negative.to_csv('log2FC_negative_data.csv', sep='\t', na_rep="NA")